## Run example workflow with user input

This notebook wraps `leadfinder/core/run_example.py` behavior in a simple Gradio UI so you can run searches interactively.

In [4]:
import json
import os
import uuid
from pathlib import Path

import gradio as gr
from dotenv import load_dotenv

from leadfinder.core.workflow import WorkflowRunner
from leadfinder.core.workflow_types import WorkflowContext
from leadfinder.core.nodes.planner import RequestPlannerNode
from leadfinder.core.nodes.anchors import AnchorGeneratorNode
from leadfinder.core.nodes.discover import DiscoverBusinessesNode
from leadfinder.core.nodes.dedupe import CanonicalizeAndDedupeNode
from leadfinder.core.nodes.score import ScoreLeadsNode
from leadfinder.core.nodes.website_socials import WebsiteSocialExtractorNode
from leadfinder.core.nodes.assemble import AssembleResultsNode
from leadfinder.core.nodes.export import ExportGeneratorNode
from leadfinder.providers.google_places import GooglePlacesProvider, GooglePlacesConfig

In [5]:
repo_root = Path.cwd()
load_dotenv(repo_root / ".env", override=False)
load_dotenv(repo_root / "notebooks" / ".env", override=False)


def build_runner(provider: GooglePlacesProvider) -> WorkflowRunner:
    return WorkflowRunner(
        nodes=[
            RequestPlannerNode(),
            AnchorGeneratorNode(),
            DiscoverBusinessesNode(provider),
            CanonicalizeAndDedupeNode(),
            ScoreLeadsNode(),
            WebsiteSocialExtractorNode(),
            AssembleResultsNode(),
            ExportGeneratorNode(export_dir=os.getenv("EXPORT_DIR", "./exports")),
        ]
    )


async def run_search(
    query: str,
    target_count: int,
    plan: str,
    include_socials: bool,
) -> dict:
    api_key = os.getenv("GOOGLE_PLACES_API_KEY", "")
    if not api_key:
        return {"error": "GOOGLE_PLACES_API_KEY is not set"}

    req = {
        "query": query,
        "geo_scope": {"query": query},
        "target_count": target_count,
        "plan": plan,
        "options": {"include_socials": include_socials, "website_fetch_cap": 50},
    }

    cfg = GooglePlacesConfig(api_key=api_key, region_code="CA")
    async with GooglePlacesProvider(cfg) as provider:
        runner = build_runner(provider)
        ctx = WorkflowContext(search_id=str(uuid.uuid4()), request=req)
        ctx = await runner.run(ctx)

    return {
        "results": ctx.results,
        "export_paths": ctx.export_paths,
        "errors": ctx.errors,
    }


def pretty_json(data: dict) -> str:
    return json.dumps(data, indent=2, ensure_ascii=False, default=str)


async def on_submit(
    query: str,
    target_count: int,
    plan: str,
    include_socials: bool,
) -> str:
    data = await run_search(
        query=query,
        target_count=target_count,
        plan=plan,
        include_socials=include_socials,
    )
    return pretty_json(data)


In [3]:
with gr.Blocks() as demo:
    gr.Markdown("# LeadFinder Workflow Runner")
    query = gr.Textbox(label="Query", value="coffee shop")
    target_count = gr.Dropdown(
        label="Target count", choices=[20, 50, 100, 200, 300, 500], value=50
    )
    plan = gr.Dropdown(label="Plan", choices=["basic", "pro", "enterprise"], value="pro")
    include_socials = gr.Checkbox(label="Include socials", value=True)

    run_btn = gr.Button("Run Search")
    output = gr.Code(label="Results", language="json")

    run_btn.click(
        on_submit,
        inputs=[query, target_count, plan, include_socials],
        outputs=output,
    )


demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
